In [49]:
from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)

Mounted at /content/gdrive


In [5]:
!pip install pydub

In [0]:
# Imports 
from tensorflow.keras.models import load_model
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio
import os
import glob
import time
import importlib
from sklearn.preprocessing import StandardScaler
from subprocess import Popen, PIPE, STDOUT
os.system("pip install pydub")

os.chdir('/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/notebooks/audio-new')


import arffToNp
importlib.reload(arffToNp)
import subprocess
import numpy as np


# Audio API

In [0]:

# TODO: add option for soft vs hard
def predict(mp4_filepath, best_model_filepath):
    """
    Outputs:
    - A tuple with predictions for each class (positive, neutral, negative)
    """

    model = fer_model()
    model.load_model(best_model_filepath)
    return model.predict(mp4_filepath)

class audio_model:
    def __init__(self):
        self.model = ()
        return

    def predict(self, mp4_filepath):
        self.preprocess(mp4_filepath)
        X = cv2.imload("test/happy.jpg")
        X = cv2.resize(X, (48,48))
        X = cv2.cvtColor(X, cv2.COLOR_BGR2GRAY)

        return self.model.predict(img)
        #return (0.1,0.2,0.7)

    def load_model(self, best_model_filepath):
        self.model = load_model(best_model_filepath)
        return

    def train(self, X_train , y_train):


         # train the model
              # TO DO : Implement a recurrent network for each "picture"
        # Look into autoencoders/PCA for dim reduction

        # Reccurent network on row time steps
        inputs = keras.Input(shape=[216,128])

        recurrent_1 = keras.layers.LSTM(100, return_sequences=True, input_shape=[None, 128] , dropout=0.2 , activation='selu') #A sequence of any length with dimensions 128 (i.e. 128 columns)

        recurrent_2 = keras.layers.LSTM(32)

        # We use the row of the image at each time step. That's why we have a repeat 
        # vector layer.
        #repeat_vector = keras.layers.RepeatVector(216, input_shape=[30])
        #recurrent_3 = keras.layers.LSTM(100, return_sequences=False)
        dense_1 = keras.layers.Dense(32 , activation='selu')
        softmax = keras.layers.Dense(3 , activation='softmax')

        x = recurrent_1(inputs)
        x = recurrent_2(x)


        #x = repeat_vector(x)
        #x = recurrent_3(x)
        x = dense_1(x)


        outputs = softmax(x)

        rnn_ae = keras.Model(inputs=inputs, outputs=outputs)
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
          initial_learning_rate=1e-5,
          decay_steps=10000,
          decay_rate=0.9)
        opt = keras.optimizers.Adam(learning_rate=lr_schedule)
        rnn_ae.compile(loss='sparse_categorical_crossentropy' , optimizer=opt , metrics=['accuracy'])
        rnn_ae.fit(X_train , y_train , epochs=500 , batch_size=32)



        return rnn_ae

    def preprocess(self, mp4_filepath , target_label_path=None):

      """
      Outputs:
      - A numpy array with dimensions (m,n). 
        - m is the units in time dependent on the audio splice rate.
        - n is the number of features from the openSMILE library.
      """


      output_wav_file = mp4_filepath[-5] + 'extracted_audio.wav'
      mp4_filename = os.path.basename(mp4_filepath)
      audio_home_dir = os.path.dirname(mp4_filepath)
      os.chdir(audio_home_dir)

      # Strip the audio from video and store as .wav file
      ffmpeg_extract_audio(mp4_filepath, output_wav_file)
      !cd '$audio_home_dir' ; mkdir to_zip

      # splice the audio files into 2 seconds with 100 ms sliding window.
      # 30 kHz sampling rate
      !cd '$audio_home_dir' ; python SliceAudio.py -i *.wav -o wav -c 2  -b 2 -s 30000 -w 100 -l 2000
    
      # Zip and move files from drive to vm
      !cd '$audio_home_dir'  ; zip -r -qq to_zip.zip  to_zip ; cd '$audio_home_dir' ; mv 'to_zip.zip' '/content/' 

      # Remove the old zip folder in vm
      !cd '$audio_home_dir' ; cd to_zip ; rm *.wav  ; cd - ; rm -d to_zip

      # Inflate the zip folder in vm
      !cd '/content/' ; unzip -qq to_zip.zip 


      # OpenSMILE feature extraction
      out_fn = os.path.join('/content/openSmile-features.arff')
      os.chdir('/content/to_zip/')
      aligned_files = glob.glob('*.wav')
      os.chdir('/content/')
      for in_fn in aligned_files:
        in_fn = os.path.join('/content/to_zip/' , in_fn)
        name = os.path.basename(in_fn)
        !cd 'opensmile-2.3.0' ; inst/bin/SMILExtract -C config/IS13_ComParE.conf -I '$in_fn' -O '$out_fn' -N $name

      # Convert .arff to .csv
      all_timepoints_feature_array = arffToNp.convert(out_fn , '')
      print("The shape of the feature matrix for one \n video is: " , all_timepoints_feature_array.shape)

      # Clean up            
      !cd to_zip ; rm *.wav ; cd - ; rm -d to_zip      
      os.remove(out_fn)
      !cd '$audio_home_dir' ; rm *.wav ; rm *.csv
      !rm to_zip.zip
      !rm *.csv
      !rm *.arff


      # Standardize
      scaler = StandardScaler()
      all_timepoints_feature_array = scaler.fit_transform(all_timepoints_feature_array)


      # Get the Y values 
      target = None 
      if target_label_path is not None:
        target_labels = np.genfromtxt(target_label_path , delimiter = ' ' , dtype='str')
        target_index = np.where(target_labels[: , 0] == mp4_filename[:-4])[0]
        target = int(target_labels[: , 1][target_index])



      return all_timepoints_feature_array , target


      # Read in each video file and add the (m,n) feature matrix to a 3D array

    def get_feature_batch(self, input_files_dir , batch_size=3000 , target_label_path=None):
        """
        Inputs: 
        - Path to the .mp4 files
        Outputs:
        - An ndarray with dims (s , m , n)
          - s is the number of samples
          - m is the number of slices for that sample (32)
          - n is the number of features (6373)
        """

        output_x = []
        output_y = None

        if target_label_path is not None:
          output_y = []


        counter = 0

        fileList = glob.glob(input_files_dir + '*.mp4')

        print(fileList)

      
        for file_path in fileList:
         
          print(file_path)

          one_sample_feat_matrix , y = self.preprocess(file_path , target_label_path=target_label_path)
          output_x.append(one_sample_feat_matrix)
          if target_label_path is not None:            
            output_y.append(y)

          if counter >= batch_size:
            break

          counter += 1

        output_y = np.asarray(output_y)
        output_y -= 1

        return output_x , output_y

def installOpenSMILE():
    """
    You must upload your downloaded version of openSMILE from the site to 
    cloud.

    """
    os.chdir('/content/')
    !tar -zxvf 'opensmile-2.3.0.tar.gz'
    !sed -i '117s/(char)/(unsigned char)/g' opensmile-2.3.0/src/include/core/vectorTransform.hpp
    !sudo apt-get update
    !sudo apt-get install autoconf automake libtool m4 gcc
    !cd 'opensmile-2.3.0' ; bash buildStandalone.sh



In [0]:
installOpenSMILE()


In [0]:
!cd 'opensmile-2.3.0' ; inst/bin/SMILExtract -h

In [0]:
audio_model_1 = audio_model()
target_path =  '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/datasets/emotiw/Val_labels.txt'
output_arr , y = audio_model_1.preprocess(mp4_filepath='/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/notebooks/audio-new/1_1.mp4' , target_label_path=target_path)

In [0]:
!cd '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/notebooks/audio-new/'; ls 

1_1.mp4				  audio.ipynb		  openSMILEShell.sh
1extracted_audio.wav		  audio.py		  __pycache__
arffToNp.py			  opensmile-2.3.0	  SliceAudio.py
audio-deep-learning-models.ipynb  opensmile-2.3.0.tar.gz  to_zip


In [0]:
output_arr.shape

(32, 6373)

# Set up Processing Pipeline


In [0]:
# Imports

import tensorflow as tf
from tensorflow import keras


/bin/bash: line 0: cd: to_zip: No such file or directory
rm: cannot remove 'to_zip': No such file or directory


In [0]:
# Preprocessing Pipeling
home_dir = '/content/'
!cd '$home_dir'; mkdir train_vids ; mkdir val_vids
train_path = '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/datasets/emotiw/Train.zip'
train_target_path =  '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/datasets/emotiw/Train_labels.txt'

val_path = '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/datasets/emotiw/Val.zip'
val_target_path =  '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/datasets/emotiw/Val_labels.txt'

vm_train_path = '/content/train_vids'
vm_val_path = '/content/val_vids'
slice_audio_path = '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/notebooks/audio-new/SliceAudio.py'


# Copy files to vm and inflate
!cp '$train_path' '$vm_train_path' ; cd '$vm_train_path' ; unzip -qq 'Train.zip'
!cp '$val_path' '$vm_val_path' ; cd '$vm_val_path' ; unzip -qq 'Val.zip'



In [0]:
 !cd '$vm_val_path' ; unzip 'Val.zip'


In [43]:
inflated_train_path = '/content/train_vids/Train/'
inflated_val_path = '/content/train_vids/Val/'

# Copy SliceAudio to dirs
!cp '$slice_audio_path' '$inflated_train_path' ; cp '$slice_audio_path' '$inflated_val_path'

cp: cannot create regular file '/content/train_vids/Val/': Not a directory


In [0]:
audio_model_test = audio_model()
      
X , y = audio_model_test.get_feature_batch(inflated_train_path , batch_size=3 , target_label_path=train_target_path)

In [69]:
#final_array_1 = np.asarray(batched_array , dtype='float')
#!cd '/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/notebooks/audio-new/arrays'
X = np.asarray(X , dtype = 'float32')
print(X.shape)
print(y.shape)

[1 2 1 2]


In [0]:
import pickle
with open('/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/Challenge-Data/emotiw/models/audio.train-opensmile-standardized_X.all.pickle', 'wb') as f:
    pickle.dump(X, f)
with open('/content/gdrive/My Drive/Machine-Learning-Projects/cs231n/Challenge-Data/emotiw/models/audio.train-opensmile-standardized_y.all.pickle', 'wb') as f:
    pickle.dump(y, f)